Sailors Excersice 

Create the database

In [2]:
import sqlite3 
import pandas as pd

In [31]:
conn = sqlite3.connect('sailing_club.db')
cursor = conn.cursor()
cursor.execute('''DROP TABLE SAILORS''')
cursor.execute('''DROP TABLE BOATS''')
cursor.execute('''DROP TABLE RESERVES''')
cursor.execute('''DROP TABLE CAPTAINS''')
conn.commit()
conn.close

<function Connection.close()>

In [2]:
conn = sqlite3.connect('sailing_club.db')
cursor = conn.cursor()


cursor.execute('''
create table SAILORS(
    Sname       varchar(15),
    SID         varchar(5) PRIMARY KEY,
    Rating      int(4),
    Age         int(4)
)
''')

cursor.execute('''
create table CAPTAINS(
    Sname       varchar(15),
    SID         varchar(5) PRIMARY KEY,
    Rating      int(4),
    Age         int(4)
)
''')


cursor.execute('''
create table BOATS(
    Bname       varchar(15),
    BID         varchar(5) PRIMARY KEY,
    Fee         int(4),
    Location    varchar(15)
)
''')

cursor.execute('''
create table RESERVES(
    SID         varchar(5),
    BID         varchar(5),
    Day         varchar(20),
    Deposit     float(7),
    FOREIGN KEY (SID) REFERENCES SAILORS(SID),
    FOREIGN KEY (BID) REFERENCES BOATS(BID)
)
''')

sailors_data = [
    ('Marx', 23, 8, 52),
    ('Martin', 25, 9, 51),
    ('Adams', 27, 8, 36),
    ('Carrey', 33, 10, 22)
]

cursor.executemany('''
INSERT INTO Sailors (Sname, SID, Rating, Age) VALUES (?, ?, ?, ?)
''', sailors_data)

boats_data = [
    ('Wayfarer', 109, 120, 'Hout Bay'),
    ('SeaPride', 108, 500, 'Fish Hoek'),
    ('Yuppie', 101, 400, 'Hout Bay'),
    ('Joy', 104, 200, 'Hout Bay')
]

cursor.executemany('''
INSERT INTO Boats (Bname, BID, Fee, Location) VALUES (?, ?, ?, ?)
''', boats_data)

reserves_data = [
    (23, 109, '01/08/2014', 120),
    (23, 108, '08/08/2014', 0),
    (25, 101, '08/08/2014', 0),
    (27, 101, '09/08/2014', 100),
    (27, 109, '04/09/2014', 0),
    (33, 109, '04/09/2014', 0),
    (33, 104, '11/09/2014', 0)
]

cursor.executemany('''
INSERT INTO Reserves (SID, BID, Day, Deposit) VALUES (?, ?, ?, ?)
''', reserves_data)

conn.commit()
conn.close

<function Connection.close()>

1. Get everything in the Sailors table

In [3]:
conn = sqlite3.connect('sailing_club.db')
cursor = conn.cursor()

query = '''
SELECT *
FROM SAILORS
'''
result_df = pd.read_sql_query(query, conn)


conn.close()

print(result_df.to_string(index=True))

    Sname SID  Rating  Age
0    Marx  23       8   52
1  Martin  25       9   51
2   Adams  27       8   36
3  Carrey  33      10   22


Get sailor ID, rank & age of all sailors, ordered from highest to lowest rank. Rank is 10 rimes rating

In [4]:
conn = sqlite3.connect('sailing_club.db')
cursor = conn.cursor()

query = '''
SELECT SID, 10*Rating AS Rank, Age
FROM SAILORS
ORDER BY Rank
'''
result_df = pd.read_sql_query(query, conn)


conn.close()

print(result_df.to_string(index=True))

  SID  Rank  Age
0  23    80   52
1  27    80   36
2  25    90   51
3  33   100   22


3. Get alphabetical list of sailors with rating less than 10

In [5]:
conn = sqlite3.connect('sailing_club.db')
cursor = conn.cursor()

query = '''
SELECT Sname, Rating
FROM SAILORS 
WHERE Rating < 10 
ORDER BY Sname 
'''
result_df = pd.read_sql_query(query, conn)


conn.close()

print(result_df.to_string(index=True))

    Sname  Rating
0   Adams       8
1  Martin       9
2    Marx       8


4. Find how much deposit money there is in total and how many tuples are in the reserves table

In [6]:
conn = sqlite3.connect('sailing_club.db')
cursor = conn.cursor()

query = '''
SELECT COUNT(Deposit) AS TotalTuples, SUM(Deposit) AS TotalMoney 
FROM RESERVES 
 '''
result_df = pd.read_sql_query(query, conn)


conn.close()

print(result_df.to_string(index=True))

   TotalTuples  TotalMoney
0            7       220.0


5. Get all info on boats in Fishoek but I'm not sure how you spell Fishoek. 

In [7]:
conn = sqlite3.connect('sailing_club.db')
cursor = conn.cursor()

query = '''
SELECT *
FROM BOATS 
WHERE LOCATION LIKE "_IS%K"
'''
result_df = pd.read_sql_query(query, conn)


conn.close()

print(result_df.to_string(index=True))

      Bname  BID  Fee   Location
0  SeaPride  108  500  Fish Hoek


6. In what Locations are boats kept? 

In [10]:
conn = sqlite3.connect('sailing_club.db')
cursor = conn.cursor()

query = '''
SELECT DISTINCT(Location) 
FROM BOATS 
'''
result_df = pd.read_sql_query(query, conn)


conn.close()

print(result_df.to_string(index=True))

    Location
0   Hout Bay
1  Fish Hoek


7. Get the names of all Boats tht have a fee value recorded in the database

In [3]:
conn = sqlite3.connect('sailing_club.db')
cursor = conn.cursor()

query = '''
SELECT DISTINCT(Bname)
FROM BOATS 
JOIN RESERVES ON RESERVES.BID = BOATS.BID   
'''
result_df = pd.read_sql_query(query, conn)


conn.close()

print(result_df.to_string(index=True))

      Bname
0  Wayfarer
1  SeaPride
2    Yuppie
3       Joy


8. Get ID of all boats that have not been reserved 

In [4]:
conn = sqlite3.connect('sailing_club.db')
cursor = conn.cursor()

query = '''
SELECT DISTINCT(Bname)
FROM BOATS  
WHERE BID NOT IN (SELECT DISTINCT(BID) FROM RESERVES)

'''
result_df = pd.read_sql_query(query, conn)


conn.close()

print(result_df.to_string(index=True))

Empty DataFrame
Columns: [Bname]
Index: []


9. Get all reservation info, including all details on the boats being reserved.

In [7]:
conn = sqlite3.connect('sailing_club.db')
cursor = conn.cursor()

query = '''
SELECT *
FROM RESERVES 
JOIN BOATS ON BOATS.BID = RESERVES.BID
'''
result_df = pd.read_sql_query(query, conn)


conn.close()

print(result_df.to_string(index=True))

  SID  BID         Day  Deposit     Bname  BID  Fee   Location
0  23  109  01/08/2014    120.0  Wayfarer  109  120   Hout Bay
1  23  108  08/08/2014      0.0  SeaPride  108  500  Fish Hoek
2  25  101  08/08/2014      0.0    Yuppie  101  400   Hout Bay
3  27  101  09/08/2014    100.0    Yuppie  101  400   Hout Bay
4  27  109  04/09/2014      0.0  Wayfarer  109  120   Hout Bay
5  33  109  04/09/2014      0.0  Wayfarer  109  120   Hout Bay
6  33  104  11/09/2014      0.0       Joy  104  200   Hout Bay


10. For all reservation, get the name of the sailor, along with the day and name of the boat booked.

In [3]:
conn = sqlite3.connect('sailing_club.db')
cursor = conn.cursor()

query = '''
SELECT RESERVES.SID, RESERVES.BID, RESERVES.Day, RESERVES.Deposit, BOATS.Bname, SAILORS.Sname
FROM RESERVES 
JOIN BOATS ON RESERVES.BID = BOATS.BID 
JOIN SAILORS ON RESERVES.SID = SAILORS.SID 
'''
result_df = pd.read_sql_query(query, conn)


conn.close()

print(result_df.to_string(index=True))

  SID  BID         Day  Deposit     Bname   Sname
0  23  109  01/08/2014    120.0  Wayfarer    Marx
1  23  108  08/08/2014      0.0  SeaPride    Marx
2  25  101  08/08/2014      0.0    Yuppie  Martin
3  27  101  09/08/2014    100.0    Yuppie   Adams
4  27  109  04/09/2014      0.0  Wayfarer   Adams
5  33  109  04/09/2014      0.0  Wayfarer  Carrey
6  33  104  11/09/2014      0.0       Joy  Carrey


11. Get the average deposit paid for each boat.

In [11]:
conn = sqlite3.connect('sailing_club.db')
cursor = conn.cursor()

query = '''
SELECT BOATS.Bname,RESERVES.BID , AVG(Deposit) AS Average_Deposit
FROM RESERVES 
JOIN BOATS ON RESERVES.BID = BOATS.BID
GROUP BY RESERVES.BID 


'''
result_df = pd.read_sql_query(query, conn)


conn.close()

print(result_df.to_string(index=True))

      Bname  BID  Average_Deposit
0    Yuppie  101             50.0
1       Joy  104              0.0
2  SeaPride  108              0.0
3  Wayfarer  109             40.0


Get the average deposit paid for each boat

In [10]:
conn = sqlite3.connect('sailing_club.db')
cursor = conn.cursor()

query = '''
SELECT

'''
result_df = pd.read_sql_query(query, conn)


conn.close()

print(result_df.to_string(index=True))

   BID  AVG(Deposit)
0  101          50.0
1  104           0.0
2  108           0.0
3  109          40.0


In [ ]:
conn = sqlite3.connect('sailing_club.db')
cursor = conn.cursor()

query = '''

'''
result_df = pd.read_sql_query(query, conn)


conn.close()

print(result_df.to_string(index=True))

In [ ]:
conn = sqlite3.connect('sailing_club.db')
cursor = conn.cursor()

query = '''

'''
result_df = pd.read_sql_query(query, conn)


conn.close()

print(result_df.to_string(index=True))

In [ ]:
conn = sqlite3.connect('sailing_club.db')
cursor = conn.cursor()

query = '''

'''
result_df = pd.read_sql_query(query, conn)


conn.close()

print(result_df.to_string(index=True))

In [ ]:
conn = sqlite3.connect('sailing_club.db')
cursor = conn.cursor()

query = '''

'''
result_df = pd.read_sql_query(query, conn)


conn.close()

print(result_df.to_string(index=True))

In [ ]:
conn = sqlite3.connect('sailing_club.db')
cursor = conn.cursor()

query = '''

'''
result_df = pd.read_sql_query(query, conn)


conn.close()

print(result_df.to_string(index=True))

In [ ]:
conn = sqlite3.connect('sailing_club.db')
cursor = conn.cursor()

query = '''

'''
result_df = pd.read_sql_query(query, conn)


conn.close()

print(result_df.to_string(index=True))

In [ ]:
conn = sqlite3.connect('sailing_club.db')
cursor = conn.cursor()

query = '''

'''
result_df = pd.read_sql_query(query, conn)


conn.close()

print(result_df.to_string(index=True))

In [ ]:
conn = sqlite3.connect('sailing_club.db')
cursor = conn.cursor()

query = '''

'''
result_df = pd.read_sql_query(query, conn)


conn.close()

print(result_df.to_string(index=True))